## <a href='https://leetcode.com/problems/number-of-longest-increasing-subsequence/'>673. Number of Longest Increasing Subsequence</a>
https://leetcode.com/articles/a-recursive-approach-to-segment-trees-range-sum-queries-lazy-propagation/
https://cp-algorithms.com/sequences/longest_increasing_subsequence.html#toc-tgt-9

Given an integer array nums, return the number of longest increasing subsequences.

__Example 1:__
```
Input: nums = [1,3,5,4,7]
Output: 2
Explanation: The two longest increasing subsequences are [1, 3, 4, 7] and [1, 3, 5, 7].
```
__Example 2:__
```
Input: nums = [2,2,2,2,2]
Output: 5
Explanation: The length of longest continuous increasing subsequence is 1, and there are 5 subsequences' length is 1, so output 5.
```
__Constraints:__

* 1 <= nums.length <= 2000
* -106 <= nums[i] <= 106

In [6]:
public class SolutionRec {
    private int answer = 0, nums[], max = 0, n = 0;
    private Map<Integer, Integer> map = new HashMap<>();
    public int solve(int[] nums) {
        this.nums = nums;
        n = nums.length;
        helper(0, -1, 1);
        return answer;
    }
    private void helper(int start, int prev, int length) {
        if (start >= n) {
            if (length > max) {
                max = length;
                answer = 0;
            }
            if (length == max) answer++;
            return;
        }
        if (prev == -1 || nums[prev] < nums[start]) helper(start + 1, start, length + 1);
        helper(start + 1, prev, length);
    }
}

In [7]:
new SolutionRec().solve(new int[] {2,2,2,2,2});

5

In [8]:
new SolutionRec().solve(new int[] {1,3,2,4,5});

2

In [17]:
public class SolutionDp {
    public int solve(int[] nums) {
        int n = nums.length;
        int[] lengths = new int[n];
        int[] counts = new int[n];
        Arrays.fill(lengths, 1);
        Arrays.fill(counts, 1);
        //     *
        // 1 3 2 4 5
        // ^   
        // 1 1 1 1 1
        for (int j = 0; j < n; ++j) {
            for (int i = 0; i < j; ++i) {
                if (nums[i] >= nums[j]) continue;
                if (lengths[i] >= lengths[j]) {
                    lengths[j] = lengths[i] + 1;
                    counts[j] = counts[i];
                } else if (lengths[i] + 1 == lengths[j]) {
                    counts[j] += counts[i];
                }
            }
        }
        int longest = 0, answer = 0;
        for (int length : lengths) longest = Math.max(length, longest);
        for (int i = 0; i < n; ++i) {
            if (lengths[i] == longest) answer += counts[i];
        }
        return answer;
    }
}

In [18]:
new SolutionDp().solve(new int[] {2,2,2,2,2});

5

In [19]:
new SolutionDp().solve(new int[] {1,3,2,4,5});

2

In [20]:
new SolutionDp().solve(new int[] {1,3,5,4,7});

2

In [27]:
public class SolutionSegTree {
    public int solve(int[] nums) {
        int n = nums.length;
        int min = nums[0], max = nums[0];
        for (int num : nums) {
            min = Math.min(num, min);
            max = Math.max(num, max);
        }
        Node root = new Node(min, max);
        for (int num : nums) {
            Value val = query(root, num - 1);
            insert(root, num, new Value(val.len + 1, val.count));
        }
        return root.val.count;
    }
    private Value query(Node node, int key) {
        if (node.rangeRight <= key) return node.val;
        else if (node.rangeLeft > key) return new Value(0, 1);
        else return merge(query(node.getLeft(), key), query(node.getRight(), key));
    }
    private void insert(Node node, int key, Value val) {
        if (node.rangeLeft == node.rangeRight) {
            node.val = merge(val, node.val);
        } else {
            if (key <= node.getMid()) {
                insert(node.getLeft(), key, val);
            } else {
                insert(node.getRight(), key, val);
            }
            node.val = merge(node.getLeft().val, node.getRight().val);
        }
    }
    private Value merge(Value v1, Value v2) {
        if (v1.len == v2.len) {
            if (v1.len == 0) return new Value(0, 1);
            return new Value(v1.len, v1.count + v2.count);
        }
        return v1.len > v2.len ? v1 : v2;
    }
    private class Node {
        public int rangeLeft, rangeRight;
        public Node left, right;
        public Value val = new Value(0, 1);
        public Node (int rangeLeft, int rangeRight) {
            this.rangeLeft = rangeLeft;
            this.rangeRight = rangeRight;
        }
        public int getMid() {
            return rangeLeft + (rangeRight - rangeLeft) / 2;
        }
        public Node getLeft() {
            if (left == null) left = new Node(rangeLeft, getMid());
            return left;
        }
        public Node getRight() {
            if (right == null) right = new Node(getMid() + 1, rangeRight);
            return right;
        }
        public void printLeaves() {
            LinkedList<Node> queue = new LinkedList<>();
            queue.offer(this);
            while (!queue.isEmpty()) {
                Node cur = queue.poll();
                if (cur.left == null && cur.right == null) {
                    System.out.println(cur);
                } else {
                    if (cur.left != null) queue.offer(cur.left);
                    if (cur.right != null) queue.offer(cur.right);
                }
            }
        }        
    }
    private class Value {
        public int len, count;
        public Value(int len, int count) {
            this.len = len;
            this.count = count;
        }
    }
}

In [28]:
new SolutionSegTree().solve(new int[] {1,3,2,4,5});

2

In [29]:
new SolutionSegTree().solve(new int[] {2,2,2,2,2});

5